# step1. 安裝CKIP

In [1]:
#!pip install -U ckiptagger[tfgpu,gdown]

In [1]:
# import os
# from pathlib import Path
# from collections import Counter
# import re
# import pickle
# from string import punctuation
# import datetime

# from gensim.models.word2vec import Word2Vec
# import numpy as np
# import pandas as pd
# from ckiptagger import data_utils, construct_dictionary, WS
# from sklearn.metrics.pairwise import cosine_similarity
# from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer, CountVectorizer
# #from google.colab import drive

In [2]:
from pathlib import Path

import pandas as pd

from model_building import word2vec_model
from split_rule_V1 import split_rule

In [3]:
root_path = Path('/Users/zoe/Documents/GitHub/20201107_d4sg_corpus_mapping')
ckip_path = root_path / 'data/ckip_model/data'
parsed_column = '缺失內容'
# dictionary
dict_path = root_path / 'dictionary'
legal_name_file = dict_path / 'name_of_legal.txt'
word_file = dict_path / 'oth_words.txt'
punc_file = dict_path / 'punctuation.pkl'
split_rule_kw_file = dict_path / 'split_rule_words.txt'
# output file name
output_path = root_path / 'output/exclude_rule_ver'
# without rule exclusion
step1_file = output_path / 'data_etl_step1_exclude_law.xlsx'  # output as excel to avoid comma parse error
step2_file = output_path / 'data_etl_step2.csv'
step3_file = output_path / 'data_etl_step3_noPuncDict.csv'
w2v_model_file = output_path / 'w2v_var.pkl'

In [5]:
file_path = step3_file

df = pd.read_csv(file_path)
df = df[df[parsed_column].notna()]
# Replace '@' with ' ' in original dataframe
df.token = df.token.apply(lambda text: text.replace('@', ' '))

AttributeError: 'float' object has no attribute 'replace'

In [17]:
for x in df.itertuples():
    try:
        x.token.replace('@', '')
    except Exception as e:
        print(x)
        break

Pandas(Index=15001, 流水號=202005150, 階段=2, 法規編號=36, 法規名稱='採購評選委員會審議規則', 條='11', 條之=nan, 項='1', 款=nan, 缺失內容='【', 法令依據=nan, 事實=nan, 改進建議=nan, 法令依據_cloud="['項次26】依採購評選委員會審議規則第11條第1項規定：「本委員會會議紀錄，應記載下列事項：一、採購案名稱。二、會議次別。三、會議時間。四、會議地點。五、主席姓名。六、出席及請假委員姓名。七、列席人員姓名。八、記錄人員姓名。九、報告事項之案由及決定。一○、討論事項之案由及決議。一一、臨時動議之案由及決議。一二、其他應行記載之事項｣，惟查本採購案108年12月11日上午9時30分之評選委員會議紀錄未記載次別，爾後請注意改正。']", _14='【', 法令_cloud="['項次', '採購評選委員會審議規則']", token=nan)


In [18]:
df.columns

Index(['流水號', '階段', '法規編號', '法規名稱', '條', '條之', '項', '款', '缺失內容', '法令依據', '事實',
       '改進建議', '法令依據_cloud', '事實&改進建議', '法令_cloud', 'token'],
      dtype='object')

# filter duplicated words

In [8]:
test = split_rule(raw_df)

FileNotFoundError: [Errno 2] File b'./dictionary/rule.csv' does not exist: b'./dictionary/rule.csv'

In [31]:
with open('/Users/zoe/Documents/GitHub/20201107_d4sg_corpus_mapping/dictionary/raw_words.txt', 'r') as f:
    s = f.readlines()

In [33]:
from collections import Counter

with open(root_path / 'dictionary/oth_word.txt', 'w', encoding='big5') as f:
     f.writelines([ x[0] for x in Counter(s).most_common()])

# step2. 設定 GPU 加速斷詞速度。執行階段 > 變更執行階段類型

In [2]:
# word cut package data path and exec path
root_path = Path('/Users/zoe/Documents/GitHub/20201107_d4sg_corpus_mapping')
ckip_path = root_path / 'data/ckip_model/data'
parsed_column = '缺失內容'
# dictionary
dict_path = root_path / 'dictionary'
legal_name_file = dict_path / 'name_of_legal.txt'
word_file = dict_path / 'oth_words.txt'
punc_file = dict_path / 'punctuation.pkl'
split_rule_kw_file = dict_path / 'split_rule_words.txt'
# output file name
output_path = root_path / 'output'
step1_file = output_path / 'data_etl_step1.csv'
step2_file = output_path / 'data_etl_step2_noPunc.csv'
step3_file = output_path / 'data_etl_step3_noPuncDict.csv'
w2v_model_file = output_path / 'w2v_var.pkl'

In [3]:
raw_df = pd.read_excel(root_path / 'data/200801至202008缺失類型(法規分段例).xlsx', sheet_name='法規')

In [4]:
split_rule(raw_df, legal_name_file, split_rule_kw_file)

,流水號,階段,法規編號,法規名稱,條,條之,項,款,缺失內容,法令依據,事實,改進建議,法令依據_cloud,事實&改進建議,法令_cloud
0,200800001,1,250,其他非屬採購法相關法令之缺失,2,NaN,NaN,NaN,(1)\t依政府採購法第30條第1項第1款勞務採購，得免收押標金、保證金之規定。本案押標金1...,NaN,NaN,NaN,[],(1)\t依政府採購法第30條第1項第1款勞務採購，得免收押標金、保證金之規定。本案押標金1...,[政府採購法]
1,200801001,8,250,其他非屬採購法相關法令之缺失,6,1,NaN,NaN,評選總表未載明受評廠商標價、本委員會全部委員姓名、職業等，未符「採購評選委員會審議規則」第6...,NaN,NaN,NaN,[],評選總表未載明受評廠商標價、本委員會全部委員姓名、職業等，未符「採購評選委員會審議規則」第6...,[採購評選委員會審議規則]
2,200801001,8,250,其他非屬採購法相關法令之缺失,3,NaN,NaN,NaN,工作小組初審意見應依「採購評選委員會審議規則」第3條規定載明工作小組姓名、職稱及專長、受評廠...,NaN,NaN,NaN,[],工作小組初審意見應依「採購評選委員會審議規則」第3條規定載明工作小組姓名、職稱及專長、受評廠...,[採購評選委員會審議規則]
3,200801001,8,1,政府採購法,NaN,NaN,NaN,NaN,「臺北市政府採購稽核小組稽核監督事項紀錄表」(三)採最有利標之核准情形(含未達公告金額)2、...,NaN,NaN,NaN,[],「臺北市政府採購稽核小組稽核監督事項紀錄表」(三)採最有利標之核准情形(含未達公告金額)2、...,[]
4,200801001,8,1,政府採購法,NaN,NaN,NaN,NaN,「臺北市政府採購稽核小組稽核監督事項紀錄表」(三)採最有利標之核准情形(含未達公告金額)2、...,NaN,NaN,NaN,[],「臺北市政府採購稽核小組稽核監督事項紀錄表」(三)採最有利標之核准情形(含未達公告金額)2、...,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23222,202007266,1,251,政府採購法施行細則,43,NaN,NaN,NaN,【第4項】本採購案招標，於109年2月6日辦理更正公告，截止投標日期為109年2月12日，惟...,NaN,NaN,NaN,[政府採購法施行細則第43條規定略以：「機關於招標文件規定廠商得請求釋疑之期限，至少應有等標...,【第4項】本採購案招標，於109年2月6日辦理更正公告，截止投標日期為109年2月12日，惟...,"[施行細則, 政府採購法]"
23223,202007273,1,114,行政疏失,NaN,NaN,NaN,NaN,項次15-查本案之評審須知係貴校自行擬定，核有規定內容過於簡略之情形(例如：廠商出席人數、廠...,NaN,NaN,NaN,[],項次15-查本案之評審須知係貴校自行擬定，核有規定內容過於簡略之情形(例如：廠商出席人數、廠...,[項次]
23224,202008303,1,15,中央機關未達公告金額採購招標辦法,2,NaN,1,3,【項次8】依中央機關未達公告金額採購招標辦法第2條第1項第3款規定略以：「……公開於主管機關...,NaN,NaN,NaN,[項次8】依中央機關未達公告金額採購招標辦法第2條第1項第3款規定略以：「……公開於主管機關...,【及第3條規定略以：「……未能取得三家以上廠商之書面報價或企劃書者，得經機關首長或其授權人員...,"[項次, 中央機關未達公告金額採購招標辦法, 同辦法]"
23225,202008303,1,251,政府採購法施行細則,6,NaN,1,3,【項次1】依政府採購法施行細則第6條第1項第3款規定：「招標文件含有選購或後續擴充項目者，應...,NaN,NaN,NaN,[項次1】依政府採購法施行細則第6條第1項第3款規定：「招標文件含有選購或後續擴充項目者，應...,【，及，查本案招標公告「後續擴充」欄載明略以：「......本案之監造技術服務自109年2月...,"[施行細則, 項次, 政府採購法]"


In [7]:
#data_utils.download_data_gdown(ckip_path)
#ws = WS(ckip_path)

#os.environ["CUDA_VISIBLE_DEVICES"] = "0"
#ws = WS("./data", disable_cuda=False)

# step3. 將檔案路徑設定到自己的Google Drive

In [3]:
#drive.mount('/content/drive', force_remount=True)

/Users/zoe/Desktop/20201107_d4sg_corpus_mapping


In [5]:
#採用包含法規50, 錯誤態樣50的法規, 事實, 建議拆分的版本


In [10]:
#word2vec_modle(step3_file, w2v_model_file)

In [11]:
def recommend_law(text, model_var_file):
    def text_preprocess(text):
        rule = re.compile(r'[^a-zA-Z0-9\u4e00-\u9fa5]')
        text = rule.sub(' ',str(text))
        text = re.sub(' +', '',text)
        text = ws([text], sentence_segmentation=True)
        text = [x for l in text for x in l]
        return text
    w2v_vocab, w2v_model, tfidf_feature, tfidf_text_vect, dictionary, df = pickle.load(open(model_var_file, 'rb'))
    text = text_preprocess(text)
    text_vect = np.zeros(100) # w2v size
    weight_sum = 0
    for word in text:
        if word in w2v_vocab and word in tfidf_feature:
            vec = w2v_model.wv[word]
            tf_idf = dictionary[word]*(text.count(word)/len(text))
            text_vect += (vec * tf_idf)
            weight_sum += tf_idf
    if weight_sum != 0:
        text_vect /= weight_sum
    #binding all doc vector and add one input vector(seems extend would be better)
    tmp_vect = [*tfidf_text_vect, text_vect]
    new_cos_sim = cosine_similarity(tmp_vect, tmp_vect)
    sim_score = np.sort(new_cos_sim[new_cos_sim.shape[0]-1])[::-1][1:11]
    
    tmp_top_10_law = df[['法規名稱','條','缺失內容']].iloc[np.argsort(new_cos_sim[new_cos_sim.shape[0]-1])[::-1][1:11]]
    tmp_top_10_law['similarity_score'] = [round(score*100,1) for score in sim_score]
    return tmp_top_10_law

In [27]:
starttime = datetime.datetime.now()

newtext = '開標時有作拒絕往來廠商調查'
result = recommend_law(newtext, w2v_model_file)

# calculate running time
endtime = datetime.datetime.now()
print("搜尋推薦時間: ",endtime - starttime)

搜尋推薦時間:  0:00:01.383325


In [29]:
result

,法規名稱,條,缺失內容,similarity_score
13476,政府採購法,103,項次16─依政府採購法第103條第1項，經刊登政府採購公報之拒絕往來廠商自刊登次日起1年或3...,96.5
11178,政府採購法,103,依政府採購法第103條第1項，經刊登政府採購公報之拒絕往來廠商自刊登次日起1年或3年不得3加...,95.2
13171,政府採購法,103,依政府採購法第103條第1項，經刊登政府採購公報之拒絕往來廠商自刊登次日起1年或3年不得3加...,95.2
2976,政府採購法,50,本案允許「共同投標」，開標前已查詢拒絕往來廠商名單，惟未於開標後對得標廠商之共同投標成員長泉...,93.1
12739,政府採購法,50,依政府採購法第103條第1項，經刊登政府採購公報之拒絕往來廠商自刊登次日起1年或3年不得3加...,93.1
4629,政府採購法,50,政府採購法第50條第1項明定投標廠商有第103條第1項不得3加投標或作為決標對象之情形，經機...,93.0
10749,政府採購法,50,有關本案「查詢拒絕往來廠商名單」1節，查有缺失如下，請檢討改進： A.依政府採購法第50條第...,92.4
9393,政府採購法,50,本案受稽核文件附有貴局查詢各投標廠商是否為拒絕往來廠商名單，其中「台灣野村總研諮詢顧問股份有...,92.2
10222,政府採購法,50,本案開標日為103年2月21日，惟稽核文件查詢拒絕往來廠商名單時間為103年3月18日，顯未...,92.1
6752,政府採購法,50,政府採購法第50條第1項第6款明定，投標廠商有第103條第1項不得3加投標或作為決標對象之情...,91.9


In [ ]:
#bad formate inner open
#seems can't be last line of pickle dump

In [4]:
# data_cleansing(raw_df, output_file=step1_file, text_column=parsed_column)
# get_punctuation(step1_file, output_file=punc_file, text_column=parsed_column)
# word_cut(step1_file, step3_file, punc_file, parsed_column, legal_name_file, word_file, ckip_path)
#word2vec_modle(step3_file, w2v_model_file)

/Users/zoe/Desktop/20201107_d4sg_corpus_mapping/output/data_etl_step1.csv  exported.
/Users/zoe/Desktop/20201107_d4sg_corpus_mapping/dictionary/punctuation.pkl  exported.
/Users/zoe/Desktop/20201107_d4sg_corpus_mapping/output/data_etl_step3_noPuncDict.csv  exported.
